<a href="https://colab.research.google.com/github/x1001000/livestream-blur-detection/blob/main/PQC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 掛載雲端硬碟

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

!git clone https://github.com/x1001000/livestream-blur-detection --quiet
%cd livestream-blur-detection

from dateutil import parser
git_log = !git log
dt = parser.parse(git_log[2].strip('Date:   '))
version = 'v' + f'{dt.date()}'.replace('-', '') + '.' + f'{dt.time()}'.replace(':', '')

# 主程式

In [ ]:
檢測人員 = "\u4F0A\u9686\u99AC" #@param {type:"string"}
檢測機台 = "TSM2330" #@param {type:"string"}

產品型號 = "NS011" #@param {type:"string"}
產品序號 = "22-10-5566" #@param {type:"string"}

檢測門檻 = 42 #@param {type:"integer"}
檢測秒數 = 0 #@param {type:"integer"}
#@markdown ⚠️ 秒數設為 0 可預覽串流影像，點擊串流影像可結束預覽。

print('程式版本：', version)

output_folder = f'/content/gdrive/MyDrive/amedifact/PQC/{產品型號}/{產品序號}'
import os
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

from datetime import datetime, timezone, timedelta
tzone = timezone(timedelta(hours=8))
start = now = datetime.now(tz=tzone)

import csv
csvfile = open(f'{output_folder}/{now}'[:-9]+'.csv', 'w', newline='')
writer = csv.writer(csvfile, delimiter=',')
writer.writerow(['檢測人員', 檢測人員, '檢測機台', 檢測機台])
writer.writerow(['產品型號', 產品型號, '產品序號', 產品序號])
writer.writerow(['檢測門檻', 檢測門檻, '檢測秒數', 檢測秒數])
writer.writerow(['時間戳', '檢測值', '檢測結果', '照片路徑'])

from detection import estimate_blur
import numpy as np
import cv2

import colab_webcam
from colab_webcam import video_stream, video_frame, js_to_image, overlap_to_bytes
# 啟動網路攝影機開始接收串流影像
video_stream()
overlap_img = ''
while 檢測秒數 == 0 or now - start < timedelta(seconds=檢測秒數):
    now = datetime.now(tz=tzone)
    # 從網路攝影機取得串流影像目前影格
    js_reply = video_frame(f' {產品型號}影像串流中', overlap_img)

    # 若無法取得影格則結束循環
    if not js_reply:
        break

    # 將回傳的JavaScript影像物件轉成 OpenCV BGR 格式
    img = js_to_image(js_reply["img"])
    H, W, _ = img.shape
    cv2.imwrite(f'{output_folder}/{now}'[:-9]+'.jpg', img)

    # 呼叫自定義影像即時處理函式
    blur_map, score, blurry = estimate_blur(img, 檢測門檻)
    color = (255, 0, 0) if blurry else (0, 255, 0)
    bgra = np.zeros([H, W, 4], dtype=np.uint8)
    cv2.rectangle(bgra, (1, 1), (W-2, H-2), color, 2)
    cv2.putText(bgra, f'{now}'[:-9], (2, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    cv2.putText(bgra, f'{score:.0f}', (W-40, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    bgra[:,:,3] = (bgra.max(axis = 2) > 0 ).astype(int) * 255

    # log in csv
    writer.writerow([f'{now}'[:-9], f'{score:.0f}', 'Fail' if blurry else 'Pass', f'{output_folder}/{now}'[23:-9]+'.jpg'])

    # 將覆疊影像轉換成 base64 格式
    overlap_bytes = overlap_to_bytes(bgra)

    # 更新覆疊影像到下一個影格
    overlap_img = overlap_bytes

csvfile.close()

from IPython.display import Javascript
Javascript('removeDom();')